In [2]:
import pandas as pd
import numpy as np
import branca.colormap as cm
import math
from sklearn.metrics.pairwise import haversine_distances
from sklearn.mixture import GaussianMixture   #Gaussian Mixture Model

In [3]:
import os
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'circle_map.html',
 "Dataframe'e Yeni Column Ekleme.ipynb",
 'density_colormap.html',
 'First.ipynb',
 'Traffic_in_Istanbul.ipynb',
 'trafikdensity.csv',
 'trafikdensity.zip']

In [4]:
data = pd.read_csv('trafikdensity.csv',parse_dates=True,index_col=0)

In [5]:
data

,DATE_TIME,LONGITUDE,LATITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
_id,,,,,,,,
1,2020-01-03 13:00:00,28.844604,41.113586,sxk9bk,9,78,53,105
2,2020-01-16 07:00:00,29.152222,41.009216,sxk9r3,6,64,27,68
3,2020-01-17 20:00:00,28.471069,41.042175,sxk1rz,53,96,72,11
4,2020-01-14 22:00:00,29.360962,40.816956,sxkbhn,17,128,73,216
5,2020-01-17 20:00:00,28.844604,40.970764,sxk906,16,82,55,123
6,2020-01-20 10:00:00,29.295044,41.108093,sxkcfe,68,152,93,88
7,2020-01-24 16:00:00,28.712769,40.992737,sxk3jq,6,91,49,299
8,2020-01-27 13:00:00,28.899536,41.069641,sxk99s,6,46,15,167
9,2020-01-06 20:00:00,28.822632,40.998230,sxk3pz,6,114,59,287


In [6]:
data = data.sort_values(by=['LONGITUDE','LATITUDE'])

In [7]:
data

,DATE_TIME,LONGITUDE,LATITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
_id,,,,,,,,
2420,2020-01-12 10:00:00,27.965698,40.981750,sx7chk,65,135,93,12
5852,2020-01-13 13:00:00,27.965698,40.981750,sx7chk,56,102,81,13
8587,2020-01-02 10:00:00,27.965698,40.981750,sx7chk,66,111,86,22
10692,2020-01-23 13:00:00,27.965698,40.981750,sx7chk,60,117,81,20
15807,2020-01-07 10:00:00,27.965698,40.981750,sx7chk,62,108,80,12
17240,2020-01-15 00:00:00,27.965698,40.981750,sx7chk,39,97,69,13
27733,2020-01-27 10:00:00,27.965698,40.981750,sx7chk,52,116,87,17
27875,2020-01-03 13:00:00,27.965698,40.981750,sx7chk,30,135,92,15
29824,2020-01-22 06:00:00,27.965698,40.981750,sx7chk,42,120,78,12


In [8]:
#converted to datetime object
data['DATE_TIME'] = pd.to_datetime(data['DATE_TIME'], yearfirst=True) 

In [9]:
#Ocak ayının tüm günleri için saat 07:00 ile 21:00 arasındaki veriyi aldık
data = data[data['DATE_TIME'].dt.strftime('%H:%M:%S').between('07:00:00','21:00:00')]

In [10]:
data

,DATE_TIME,LONGITUDE,LATITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
_id,,,,,,,,
2420,2020-01-12 10:00:00,27.965698,40.981750,sx7chk,65,135,93,12
5852,2020-01-13 13:00:00,27.965698,40.981750,sx7chk,56,102,81,13
8587,2020-01-02 10:00:00,27.965698,40.981750,sx7chk,66,111,86,22
10692,2020-01-23 13:00:00,27.965698,40.981750,sx7chk,60,117,81,20
15807,2020-01-07 10:00:00,27.965698,40.981750,sx7chk,62,108,80,12
27733,2020-01-27 10:00:00,27.965698,40.981750,sx7chk,52,116,87,17
27875,2020-01-03 13:00:00,27.965698,40.981750,sx7chk,30,135,92,15
34767,2020-01-08 13:00:00,27.965698,40.981750,sx7chk,62,116,87,11
42407,2020-01-17 10:00:00,27.965698,40.981750,sx7chk,54,111,83,15


In [11]:
result = data.agg({'NUMBER_OF_VEHICLES': ['mean', 'min', 'max']})

In [12]:
result

,NUMBER_OF_VEHICLES
mean,108.92987
min,11.00000
max,1216.00000


In [13]:
#yeni verilerle
cond1smaller = data['NUMBER_OF_VEHICLES'] < 94.032258
cond2bigger = data['NUMBER_OF_VEHICLES'] > 94.032258
print(data[cond1smaller].shape)

(558191, 8)


In [14]:
cond1smaller = data['NUMBER_OF_VEHICLES'] < 108.92987
cond2bigger = data['NUMBER_OF_VEHICLES'] > 108.92987
print(data[cond1smaller].shape)

(604602, 8)


In [15]:
print(data[cond2bigger].shape)

(301283, 8)


In [16]:
#Folium kütüphanesi ile haritalara belli bir koordinat verirsek mesela İstanbul haritasına ulaşabileceğiz.
!pip install folium

In [17]:
import folium

In [18]:
#İstanbul haritasını görürüz
harita = folium.Map(location=[41,29],
tiles="cartodbpositron",
width="%100",
height="%100",
zoom_start=10)

harita

In [19]:
#İskele koordinatlarını bir dataframe içinde topluyoruz.
iskeleler = pd.DataFrame({
    'lat':[40.992752,41.083202,41.173462,41.06683,41.040556,41.034828,41.040300,41.07583,41.041084,41.116490,41.044722,40.9522,41.160820,41.050990,41.107280,41.0176,41.10308742259623,41.0472,41.0309,41.0398,41.113950,41.0338,41.074234,41.1002,41.0217,41.030480,41.036910,41.07966,41.04752,41.13449,41.18198,41.16637,41.0468,41.027683,41.003487,40.973122,41.016777,40.972992,41.018234,41.010357,40.918056,40.87471,40.88665,40.814595,41.121834,40.979809,40.995644],
    'lon':[29.023165,29.066652,29.088163,29.043409,28.944722,28.949203,29.005600,29.044,29.007347,29.093240,29.044444,29.0939,29.046200,29.051450,29.080240,28.9751,29.056026935577393,28.9375,28.9525,28.9497,29.059960,28.9933,29.058338,29.06548,28.9767,28.966220,29.029710,29.06541,29.02554,29.091,29.07499,29.05683,28.94104,29.015259,28.957884,28.877699,28.976494,28.716211,28.577883,29.010363,29.128611,29.236231,29.186825,29.302454,29.070221,29.024643,29.018513],
    'name':['Kadıköy','AnadoluHisarı','AnadoluKavağı','Arnavutköy','Ayvansaray', 'Balat','Barbaros Hayrettin Paşa','Bebek','Beşiktaş','Paşabahçe','Beylerbeyi','Bostancı','Büyükdere','Çengelköy','Çubuklu','Eminönü','Emirgan','Eyüp','Fener','Hasköy','İstinye','Kabataş','Kandilli','Kanlıca','Karaköy','Kasımpaşa','Kuzguncuk','Küçüksu','Ortaköy','Beykoz','Rumeli Kavağı','Sarıyer','Sütlüce','Üsküdar','Yenikapı','Bakırköy','Sirkeci','Avcılar','Büyükçekmece','Harem','Maltepe','Pendik','Kartal','Tuzla','Yeniköy','Moda','Haydarpaşa']
}, dtype=str)

iskeleler

,lat,lon,name
0,40.992752,29.023165,Kadıköy
1,41.083202,29.066652,AnadoluHisarı
2,41.173462,29.088163,AnadoluKavağı
3,41.06683,29.043409,Arnavutköy
4,41.040556,28.944722,Ayvansaray
5,41.034828,28.949203,Balat
6,41.0403,29.0056,Barbaros Hayrettin Paşa
7,41.07583,29.044,Bebek
8,41.041084,29.007347,Beşiktaş
9,41.11649,29.09324,Paşabahçe


In [20]:
# add circles of piers one by one on the map
for i in range(0,len(iskeleler)):
   folium.Circle(
      location=[iskeleler.iloc[i]['lat'], iskeleler.iloc[i]['lon']],
      popup=iskeleler.iloc[i]['name'],
      radius=1000,  #meter
      color='#69b3a2',
      fill=True,
      fill_color='#69b3a2',
      fill_opacity=0.2
   ).add_to(harita)

# Show the map again
harita

In [21]:
#calculated distance between piers and densities
distance_matrix = haversine_distances(
    iskeleler[['lat', 'lon']],
    data[['LATITUDE', 'LONGITUDE']]
)

In [22]:
distance_matrix = distance_matrix * 6371  #multiply by Earth radius to get kilometers

In [23]:
#Her satır bir iskele. Her sütun, n inci satırdaki iskele koordinatları ile yoğunluğu verilen koordinatların uzaklığı.
print(distance_matrix)
print(distance_matrix.shape)

[[6657.80644901 6657.80644901 6657.80644901 ... 3929.64288713
  3929.64288713 3929.64288713]
 [6898.26962847 6898.26962847 6898.26962847 ... 3511.44366428
  3511.44366428 3511.44366428]
 [7022.81257332 7022.81257332 7022.81257332 ... 3302.68867627
  3302.68867627 3302.68867627]
 ...
 [6914.60439469 6914.60439469 6914.60439469 ... 3445.53992756
  3445.53992756 3445.53992756]
 [6673.63410493 6673.63410493 6673.63410493 ... 3948.67090532
  3948.67090532 3948.67090532]
 [6627.22700089 6627.22700089 6627.22700089 ... 3951.3162134
  3951.3162134  3951.3162134 ]]
(47, 905885)


In [24]:
#İlk iskelenin yoğunluklara uzaklığı. Distance matrix'i anlamak adına alttaki kodları yazdım.
print(distance_matrix[0,:])
print(distance_matrix[0].shape)
print(len(distance_matrix))
print(len(distance_matrix[0]))


[6657.80644901 6657.80644901 6657.80644901 ... 3929.64288713 3929.64288713
 3929.64288713]
(905885,)
47
905885


In [25]:
#1km'den küçük olan uzaklıkları(daire içinde olanları) burada topluyorum ve result denen bir array'e koyuyorum.

mLessThan1 = distance_matrix < 1000
#print(mLessThan1[46,:])
#print(mLessThan1.shape)
# Sum elements <1 across rows
result = np.sum(distance_matrix*mLessThan1, axis=1)
#print(result.shape)
print(result)

#result arrayi artık iskelelerin etrafındaki circle'daki yoğunlukların toplamını verir

#find max value
max = np.max(result)
print(max)

#elements of result divided by max
normalized_data = np.divide(result,max)

[1.89673584e+08 1.76277030e+08 1.07488859e+08 1.76422331e+08
 1.98628431e+08 1.96964376e+08 1.89630124e+08 1.80262907e+08
 1.89789537e+08 1.71299997e+08 1.78491556e+08 1.42396432e+08
 1.31521979e+08 1.73750869e+08 1.72191044e+08 2.00192919e+08
 1.75097086e+08 1.97892430e+08 1.97326789e+08 1.98045049e+08
 1.69749854e+08 1.92261862e+08 1.76974346e+08 1.73627921e+08
 1.99372460e+08 1.96918622e+08 1.83065309e+08 1.77043835e+08
 1.81297844e+08 1.51625816e+08 1.01768599e+08 1.20955952e+08
 1.97229440e+08 1.89071681e+08 1.93362916e+08 1.46756738e+08
 2.00278209e+08 9.66126314e+07 6.03659560e+07 1.92960328e+08
 1.21733764e+08 1.06917519e+08 1.10615763e+08 6.36994954e+07
 1.66000876e+08 1.88317699e+08 1.91626194e+08]
200278208.50040802


In [26]:
#elements of normalized_data is between 0 and 1
print(normalized_data)

[0.94705053 0.88016081 0.53669772 0.8808863  0.99176257 0.98345385
 0.94683353 0.90006251 0.94762949 0.85531021 0.89121806 0.71099314
 0.6566964  0.86754755 0.85975926 0.99957415 0.87426929 0.98808768
 0.9852634  0.98884971 0.84757027 0.95997395 0.88364254 0.86693366
 0.99547755 0.9832254  0.91405506 0.88398951 0.90523001 0.75707596
 0.50813616 0.60393965 0.98477734 0.9440452  0.96547157 0.73276438
 1.         0.48239213 0.30141051 0.96346142 0.60782331 0.53384499
 0.55231053 0.31805505 0.82885141 0.94028053 0.95680002]


In [28]:
colormap = cm.LinearColormap(colors=['green', 'yellow', 'orange', 'red'], vmin=0, vmax=1)

# add colorful circles of piers one by one on the map according to normalized_data
for i in range(0,len(iskeleler)):
   folium.Circle(
      location=[iskeleler.iloc[i]['lat'], iskeleler.iloc[i]['lon']],
      popup=iskeleler.iloc[i]['name'],
      radius=1000,  #meter
      color='#69b3a2',
      fill=True,
      fill_color=colormap(normalized_data[i]),
      fill_opacity=0.2
   ).add_to(harita)

# Show the map again
harita

In [29]:
# create a LinearColorMap and assign colors, vmin, and vmax
# the colormap will show green for 0 density all the way up to red for 500 density
colormap = cm.LinearColormap(colors=['green', 'yellow', 'orange', 'red'], vmin=0, vmax=500)

#added tiny colourful circles one by one. 
for i in range(len(data)):
    folium.Circle(
        location=[data.iloc[i]['LATITUDE'], data.iloc[i]['LONGITUDE']],
        radius=10,
        fill=True,
        color=colormap(data.iloc[i]['NUMBER_OF_VEHICLES']),
        fill_opacity=0.2
    ).add_to(harita)
    
harita.add_child(colormap)

harita.save('density_colormap.html')

In [39]:
#number of clusters = 47 (piers)
gmm = GaussianMixture(n_components=47)

X = pd.concat([data["LONGITUDE"], data["LATITUDE"]],axis=1)

X

,LONGITUDE,LATITUDE
_id,,
2420,27.965698,40.981750
5852,27.965698,40.981750
8587,27.965698,40.981750
10692,27.965698,40.981750
15807,27.965698,40.981750
27733,27.965698,40.981750
27875,27.965698,40.981750
34767,27.965698,40.981750
42407,27.965698,40.981750


In [40]:
#Fit the GMM model for the density dataset which expresses the dataset as a mixture of 47 Gaussian Distribution
gmm.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=47, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [41]:
gmm.predict(X)

array([44, 44, 44, ..., 31, 31, 31], dtype=int64)

In [42]:
z = gmm.score_samples(X) # this method returns log-likehood

In [43]:
z  #the log of the probability density function (PDF)

array([-1.39699485, -1.39699485, -1.39699485, ..., -6.29614716,
       -6.29614716, -6.29614716])

In [44]:
print(z.shape)

(905885,)


In [47]:
#3 boyutlu (3.boyut zaman)

#number of clusters = 47 (piers)
gmm = GaussianMixture(n_components=47)

#converted datetime to integer for DATE_TIME column
data['DATE_TIME'] = data['DATE_TIME'].apply(lambda x: x.value)

X = pd.concat([data["LONGITUDE"], data["LATITUDE"],data["DATE_TIME"]],axis=1)

X

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,LONGITUDE,LATITUDE,DATE_TIME
_id,,,
2420,27.965698,40.981750,1578823200000000000
5852,27.965698,40.981750,1578920400000000000
8587,27.965698,40.981750,1577959200000000000
10692,27.965698,40.981750,1579784400000000000
15807,27.965698,40.981750,1578391200000000000
27733,27.965698,40.981750,1580119200000000000
27875,27.965698,40.981750,1578056400000000000
34767,27.965698,40.981750,1578488400000000000
42407,27.965698,40.981750,1579255200000000000


In [53]:
#Fit the GMM model for the density dataset which expresses the dataset as a mixture of 47 Gaussian Distribution
gmm.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=47, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [54]:
gmm.predict(X)

array([44,  8,  0, ..., 26, 24, 13], dtype=int64)

In [55]:
y = gmm.score_samples(X) # this method returns log-likehood

In [57]:
y  #the log of the probability density function (PDF)

array([-40.36525814, -40.20651611, -39.74339556, ..., -39.12115895,
       -38.94427935, -38.64563924])